# Transformers & Attention Mechanisms

## Week 3, Day 3 - Understanding Modern NLP Architecture

### What You'll Learn:
1. **Attention Mechanism** - The breakthrough idea
2. **Self-Attention** - Query, Key, Value
3. **Multi-Head Attention** - Multiple perspectives
4. **Transformer Architecture** - Encoder-Decoder
5. **🎯 Practical**: Text Classification with BERT

---

## 1. Why Transformers?

### Problems with RNNs/LSTMs:

```
Sequential Processing:
Word 1 → Word 2 → Word 3 → Word 4 → Word 5
  ↓        ↓        ↓        ↓        ↓
Can't parallelize! Must process one at a time.
```

**Key Issues:**
- ❌ Sequential processing (slow)
- ❌ Long-range dependencies fade
- ❌ Vanishing gradients
- ❌ Can't use GPU parallelization effectively

### The Transformer Solution:

```
Parallel Processing:
Word 1 ──┐
Word 2 ──┼──→ Process ALL words simultaneously!
Word 3 ──┤
Word 4 ──┤
Word 5 ──┘
```

**Advantages:**
- ✅ Parallel processing (fast)
- ✅ Direct connections between all words
- ✅ Better long-range dependencies
- ✅ Scales with GPU power

## 2. Attention Mechanism Intuition

### The Core Idea:

When reading: **"The animal didn't cross the street because it was too tired"**

What does **"it"** refer to?

```
Attention weights:
The      → 0.01
animal   → 0.85  ← High attention!
didn't   → 0.02
cross    → 0.03
street   → 0.05
because  → 0.01
it       → 0.00
was      → 0.01
too      → 0.01
tired    → 0.01
```

**Attention helps the model focus on relevant words!**

In [ ]:
# Install required packages
# !pip install transformers datasets torch matplotlib numpy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

print(f"PyTorch version: {torch.__version__}")
print(f"GPU available: {torch.cuda.is_available()}")

## 3. Scaled Dot-Product Attention

### The Formula:

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

### Components:

| Component | Meaning |
|-----------|----------|
| **Q** (Query) | "What am I looking for?" |
| **K** (Key) | "What do I contain?" |
| **V** (Value) | "What information do I have?" |
| **$d_k$** | Dimension of keys (for scaling) |

### Analogy: Library Search

```
You (Query):     "I need books about machine learning"
Book Labels (Keys):  ["ML", "Cooking", "History", "AI"]
Book Content (Values): [ML_book, Cook_book, Hist_book, AI_book]

Attention scores: [0.6, 0.0, 0.0, 0.4]
Result: 0.6 * ML_book + 0.4 * AI_book
```

In [ ]:
def scaled_dot_product_attention(Q, K, V, mask=None):
    """
    Compute scaled dot-product attention.
    
    Args:
        Q: Query matrix (batch_size, seq_len, d_k)
        K: Key matrix (batch_size, seq_len, d_k)
        V: Value matrix (batch_size, seq_len, d_v)
        mask: Optional mask
    
    Returns:
        output: Attention output
        attention_weights: Attention weights for visualization
    """
    d_k = Q.size(-1)
    
    # Step 1: Compute attention scores
    scores = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(d_k)
    
    # Step 2: Apply mask (if provided)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    
    # Step 3: Apply softmax
    attention_weights = F.softmax(scores, dim=-1)
    
    # Step 4: Multiply by values
    output = torch.matmul(attention_weights, V)
    
    return output, attention_weights

# Test with simple example
seq_len, d_k = 5, 8
Q = torch.randn(1, seq_len, d_k)
K = torch.randn(1, seq_len, d_k)
V = torch.randn(1, seq_len, d_k)

output, attn_weights = scaled_dot_product_attention(Q, K, V)

print(f"Input shape: {Q.shape}")
print(f"Output shape: {output.shape}")
print(f"Attention weights shape: {attn_weights.shape}")
print(f"\nAttention weights (should sum to 1):\n{attn_weights[0]}")

In [ ]:
# Visualize attention weights
plt.figure(figsize=(8, 6))
plt.imshow(attn_weights[0].detach().numpy(), cmap='viridis')
plt.colorbar(label='Attention Weight')
plt.xlabel('Key Position')
plt.ylabel('Query Position')
plt.title('Attention Weight Matrix')
plt.show()

## 4. Self-Attention

### What is Self-Attention?

**Self-Attention**: Each word attends to all other words in the same sentence.

```
Sentence: "The cat sat on the mat"

For word "sat":
  - How much attention to "The"? → 0.05
  - How much attention to "cat"? → 0.60 (subject!)
  - How much attention to "sat"? → 0.10 (itself)
  - How much attention to "on"?  → 0.15
  - How much attention to "the"? → 0.05
  - How much attention to "mat"? → 0.05
```

### Key Insight:
Q, K, V all come from the **same input** (hence "self")

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        self.embed_dim = embed_dim
        
        # Linear transformations for Q, K, V
        self.query = nn.Linear(embed_dim, embed_dim)
        self.key = nn.Linear(embed_dim, embed_dim)
        self.value = nn.Linear(embed_dim, embed_dim)
        
    def forward(self, x):
        """
        Args:
            x: Input tensor (batch_size, seq_len, embed_dim)
        Returns:
            output: Attention output
            attention_weights: For visualization
        """
        # Generate Q, K, V from same input
        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)
        
        # Apply scaled dot-product attention
        output, attention_weights = scaled_dot_product_attention(Q, K, V)
        
        return output, attention_weights

# Test self-attention
embed_dim = 64
seq_len = 10
batch_size = 2

self_attn = SelfAttention(embed_dim)
x = torch.randn(batch_size, seq_len, embed_dim)

output, attn_weights = self_attn(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print(f"Attention weights shape: {attn_weights.shape}")

## 5. Multi-Head Attention

### Why Multiple Heads?

Different heads can focus on different aspects:

```
Sentence: "The quick brown fox jumps"

Head 1: Focus on syntax
  "quick" → "fox" (adjective-noun)
  
Head 2: Focus on semantics  
  "fox" → "jumps" (subject-verb)
  
Head 3: Focus on position
  Each word → nearby words
```

### Architecture:

```
Input
  ↓
┌─────┬─────┬─────┬─────┐
│Head1│Head2│Head3│Head4│  ← Multiple attention heads
└─────┴─────┴─────┴─────┘
  ↓
Concatenate
  ↓
Linear Layer
  ↓
Output
```

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        assert embed_dim % num_heads == 0, "embed_dim must be divisible by num_heads"
        
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        
        # Linear layers for Q, K, V
        self.qkv = nn.Linear(embed_dim, embed_dim * 3)
        self.out = nn.Linear(embed_dim, embed_dim)
        
    def forward(self, x):
        batch_size, seq_len, embed_dim = x.shape
        
        # Generate Q, K, V
        qkv = self.qkv(x)  # (batch, seq_len, 3*embed_dim)
        qkv = qkv.reshape(batch_size, seq_len, 3, self.num_heads, self.head_dim)
        qkv = qkv.permute(2, 0, 3, 1, 4)  # (3, batch, heads, seq_len, head_dim)
        Q, K, V = qkv[0], qkv[1], qkv[2]
        
        # Scaled dot-product attention for each head
        d_k = self.head_dim
        scores = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(d_k)
        attn_weights = F.softmax(scores, dim=-1)
        attn_output = torch.matmul(attn_weights, V)
        
        # Concatenate heads
        attn_output = attn_output.permute(0, 2, 1, 3).contiguous()
        attn_output = attn_output.reshape(batch_size, seq_len, embed_dim)
        
        # Final linear layer
        output = self.out(attn_output)
        
        return output, attn_weights

# Test multi-head attention
embed_dim = 64
num_heads = 8
seq_len = 10
batch_size = 2

mha = MultiHeadAttention(embed_dim, num_heads)
x = torch.randn(batch_size, seq_len, embed_dim)

output, attn_weights = mha(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print(f"Attention weights shape: {attn_weights.shape}")
print(f"Number of heads: {num_heads}")

## 6. Positional Encoding

### The Problem:

Attention has **no notion of order**!

```
"Dog bites man" = "Man bites dog" (to attention!)
```

### Solution: Positional Encoding

Add position information to embeddings:

$$PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d}}\right)$$
$$PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d}}\right)$$

Where:
- `pos` = position in sequence
- `i` = dimension index
- `d` = embedding dimension

In [ ]:
def positional_encoding(seq_len, d_model):
    """
    Generate positional encoding.
    
    Args:
        seq_len: Sequence length
        d_model: Embedding dimension
    
    Returns:
        Positional encoding matrix (seq_len, d_model)
    """
    position = np.arange(seq_len)[:, np.newaxis]
    div_term = np.exp(np.arange(0, d_model, 2) * -(np.log(10000.0) / d_model))
    
    pe = np.zeros((seq_len, d_model))
    pe[:, 0::2] = np.sin(position * div_term)
    pe[:, 1::2] = np.cos(position * div_term)
    
    return torch.FloatTensor(pe)

# Visualize positional encoding
seq_len = 100
d_model = 128
pe = positional_encoding(seq_len, d_model)

plt.figure(figsize=(12, 6))
plt.imshow(pe.numpy(), cmap='RdBu', aspect='auto')
plt.colorbar(label='Value')
plt.xlabel('Embedding Dimension')
plt.ylabel('Position in Sequence')
plt.title('Positional Encoding Visualization')
plt.show()

print(f"Positional encoding shape: {pe.shape}")

## 7. Transformer Block

### Complete Transformer Encoder Block:

```
Input
  ↓
Multi-Head Attention
  ↓
Add & Normalize (Residual)
  ↓
Feed-Forward Network
  ↓
Add & Normalize (Residual)
  ↓
Output
```

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout=0.1):
        super().__init__()
        
        # Multi-head attention
        self.attention = MultiHeadAttention(embed_dim, num_heads)
        
        # Feed-forward network
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, embed_dim)
        )
        
        # Layer normalization
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        # Multi-head attention with residual
        attn_output, attn_weights = self.attention(x)
        x = self.norm1(x + self.dropout(attn_output))
        
        # Feed-forward with residual
        ffn_output = self.ffn(x)
        x = self.norm2(x + self.dropout(ffn_output))
        
        return x, attn_weights

# Test transformer block
embed_dim = 64
num_heads = 8
ff_dim = 256
seq_len = 10
batch_size = 2

transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = torch.randn(batch_size, seq_len, embed_dim)

output, attn_weights = transformer_block(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print("✅ Transformer block working!")

## 8. Introduction to HuggingFace 🤗

### What is HuggingFace?

**HuggingFace** is the GitHub of AI models:
- 100,000+ pre-trained models
- Easy-to-use APIs
- State-of-the-art NLP, Vision, Audio models

### Popular Models:
- **BERT**: Text understanding
- **GPT-2/GPT-3**: Text generation
- **T5**: Text-to-text tasks
- **RoBERTa**: Improved BERT
- **DistilBERT**: Faster, smaller BERT

In [ ]:
from transformers import AutoTokenizer, AutoModel
import warnings
warnings.filterwarnings('ignore')

# Load pre-trained BERT model
model_name = "bert-base-uncased"

print(f"Loading {model_name}...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

print(f"✅ Model loaded!")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

## 9. Tokenization with Transformers

### How BERT Tokenizes:

```
Input: "Transformers are amazing!"

Tokens: [CLS] transform ##ers are amazing ! [SEP]
IDs:    [101]  [19081] [2545] [2024] [6429] [999] [102]
```

**Special Tokens:**
- `[CLS]`: Start of sequence
- `[SEP]`: Separator/End of sequence
- `[PAD]`: Padding token
- `[MASK]`: Masked token (for training)

In [ ]:
# Example tokenization
text = "Transformers revolutionized natural language processing!"

# Tokenize
tokens = tokenizer.tokenize(text)
print(f"Original text: {text}")
print(f"\nTokens: {tokens}")

# Convert to IDs
token_ids = tokenizer.encode(text)
print(f"\nToken IDs: {token_ids}")

# Decode back
decoded = tokenizer.decode(token_ids)
print(f"\nDecoded: {decoded}")

In [ ]:
# Batch tokenization with padding
sentences = [
    "I love transformers!",
    "BERT is powerful.",
    "Natural language processing has been revolutionized by attention mechanisms."
]

# Tokenize with padding and truncation
encoded = tokenizer(
    sentences,
    padding=True,
    truncation=True,
    max_length=20,
    return_tensors='pt'
)

print("Input IDs shape:", encoded['input_ids'].shape)
print("Attention mask shape:", encoded['attention_mask'].shape)
print("\nInput IDs:\n", encoded['input_ids'])
print("\nAttention Mask:\n", encoded['attention_mask'])

## 10. 🎯 Project: Sentiment Analysis with BERT

Let's build a sentiment classifier using pre-trained BERT!

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# Load IMDB dataset
print("Loading IMDB dataset...")
dataset = load_dataset("imdb", split="train[:1000]")  # Small subset for demo
test_dataset = load_dataset("imdb", split="test[:200]")

print(f"Training samples: {len(dataset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"\nExample: {dataset[0]}")

In [ ]:
# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

print("Tokenizing dataset...")
tokenized_train = dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

print("✅ Tokenization complete!")

In [ ]:
# Load BERT for classification
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2  # Binary classification (positive/negative)
)

print(f"Model loaded with {sum(p.numel() for p in model.parameters()):,} parameters")

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Metric function
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {'accuracy': accuracy_score(labels, predictions)}

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

print("✅ Trainer ready!")

In [ ]:
# Train the model
print("🚀 Starting training...")
trainer.train()
print("✅ Training complete!")

In [ ]:
# Test on custom sentences
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=-1).item()
    sentiment = "Positive 😊" if prediction == 1 else "Negative 😞"
    confidence = torch.softmax(outputs.logits, dim=-1)[0][prediction].item()
    return sentiment, confidence

# Test examples
test_sentences = [
    "This movie was absolutely fantastic! I loved every minute.",
    "Terrible film. Waste of time and money.",
    "It was okay, nothing special.",
    "Best movie I've seen this year!"
]

print("\n" + "="*60)
print("SENTIMENT ANALYSIS RESULTS")
print("="*60)

for sentence in test_sentences:
    sentiment, confidence = predict_sentiment(sentence)
    print(f"\nText: {sentence}")
    print(f"Sentiment: {sentiment} (Confidence: {confidence:.2%})")

## 11. Transformer vs RNN Comparison

| Feature | RNN/LSTM | Transformer |
|---------|----------|-------------|
| **Processing** | Sequential | Parallel |
| **Speed** | Slow | Fast |
| **Long-range dependencies** | Difficult | Easy |
| **GPU utilization** | Poor | Excellent |
| **Training time** | Long | Shorter |
| **Memory** | Lower | Higher |
| **Interpretability** | Hard | Easier (attention weights) |

### When to use what?

**Use RNN/LSTM:**
- Small datasets
- Limited compute
- Online/streaming data

**Use Transformers:**
- Large datasets
- GPU available
- State-of-the-art performance needed

## 12. Summary & Key Takeaways

### What We Learned:

✅ **Attention Mechanism**: Focus on relevant parts of input

✅ **Self-Attention**: Each word attends to all others

✅ **Multi-Head Attention**: Multiple perspectives simultaneously

✅ **Positional Encoding**: Adding position information

✅ **Transformer Architecture**: Complete encoder block

✅ **HuggingFace**: Using pre-trained models

✅ **Practical Application**: Sentiment analysis with BERT

### Key Formula:

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

### Next Steps:
- **Day 4**: GPT architecture and text generation
- **Day 5**: Fine-tuning transformers for custom tasks
- **Week 4**: Large Language Models (BERT, GPT, T5)

## 13. 📝 Practice Exercises

### Exercise 1: Visualize Attention
Create a heatmap showing which words attend to which in a sentence.

### Exercise 2: Different Datasets
Try the sentiment classifier on different datasets (tweets, reviews).

### Exercise 3: Compare Models
Compare BERT, DistilBERT, and RoBERTa on the same task.

### Exercise 4: Custom Transformer
Build a complete transformer from scratch for a simple task.

---

**End of Day 3 - Transformers & Attention Mechanisms** 🎉